In [9]:
import pandas as pd
data = pd.read_csv('traindata2')

In [10]:
#2-gram
alltext = []
for i in data['text']:
    tl = i.split(' ')
    if len(tl) > 1:
        tl2 = []
        for x in range(len(tl)-1):
            tl2.append(' '.join(tl[x:x+2]))
        tl.extend(tl2)
    alltext.append(tl)

In [11]:
from gensim.corpora import Dictionary
dct = Dictionary(alltext)

In [12]:
from keras.preprocessing import sequence
X = [dct.doc2idx(txt) for txt in alltext]
X =sequence.pad_sequences(X, padding='post', value=len(dct))

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,data.iloc[:,-1],test_size=0.3,random_state=0)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D

In [15]:
print('Build model...')
model = Sequential()
#我們從一個有效的嵌入層（embedding layer）開始，它將我們的詞彙索引（vocab indices ）映射到詞向量的維度上.
model.add(Embedding(len(dct)+1,200,input_length=X.shape[1]))
# 我們增加 GlobalAveragePooling1D, 這將平均計算文檔中所有詞彙的的詞嵌入
model.add(GlobalAveragePooling1D())
#我們投射到單個單位的輸出層上
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2673, 200)         31751400  
                                                                 
 global_average_pooling1d_1   (None, 200)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 4)                 804       
                                                                 
Total params: 31,752,204
Trainable params: 31,752,204
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x=X_train,y=y_train, validation_split=0.1, epochs=10, verbose=1) 

Epoch 1/10
9/9 [==============================] - 3s 311ms/step - loss: 1.3860 - accuracy: 0.2720 - val_loss: 1.3950 - val_accuracy: 0.1667
Epoch 2/10
9/9 [==============================] - 3s 287ms/step - loss: 1.3771 - accuracy: 0.3065 - val_loss: 1.3942 - val_accuracy: 0.1667
Epoch 3/10
9/9 [==============================] - 3s 306ms/step - loss: 1.3701 - accuracy: 0.3103 - val_loss: 1.3952 - val_accuracy: 0.1667
Epoch 4/10
9/9 [==============================] - 3s 294ms/step - loss: 1.3636 - accuracy: 0.3142 - val_loss: 1.3974 - val_accuracy: 0.1667
Epoch 5/10
9/9 [==============================] - 3s 306ms/step - loss: 1.3586 - accuracy: 0.3142 - val_loss: 1.3955 - val_accuracy: 0.1667
Epoch 6/10
9/9 [==============================] - 3s 307ms/step - loss: 1.3539 - accuracy: 0.3218 - val_loss: 1.3896 - val_accuracy: 0.1667
Epoch 7/10
9/9 [==============================] - 3s 308ms/step - loss: 1.3486 - accuracy: 0.3295 - val_loss: 1.3961 - val_accuracy: 0.1667
Epoch 8/10
9/9 [====

In [17]:
model.evaluate(X_test, y_test ,batch_size=10)

13/13 [==============================] - 0s 6ms/step - loss: 1.4098 - accuracy: 0.2080


[1.4098100662231445, 0.20800000429153442]

In [18]:
model.predict(X_test)

array([[0.33714488, 0.19642372, 0.2596902 , 0.20674114],
       [0.28811014, 0.22720993, 0.26112264, 0.22355726],
       [0.33079347, 0.19970362, 0.26128063, 0.2082223 ],
       [0.33699307, 0.1963494 , 0.25946254, 0.20719498],
       [0.32928577, 0.20140716, 0.25970602, 0.20960103],
       [0.30439752, 0.21740393, 0.25953373, 0.21866482],
       [0.3208795 , 0.20541759, 0.26262286, 0.21108004],
       [0.30832767, 0.21444146, 0.2595773 , 0.21765353],
       [0.29825908, 0.21721616, 0.26411748, 0.22040725],
       [0.3131802 , 0.21184973, 0.26069358, 0.21427643],
       [0.32643038, 0.20204711, 0.2608906 , 0.21063194],
       [0.28313813, 0.22587523, 0.26347834, 0.22750835],
       [0.33497205, 0.19793485, 0.25956845, 0.20752464],
       [0.2794012 , 0.23301493, 0.26646325, 0.22112063],
       [0.33339736, 0.19824038, 0.25989154, 0.20847082],
       [0.31149608, 0.21081954, 0.26038477, 0.21729963],
       [0.33952847, 0.19470175, 0.2597737 , 0.20599605],
       [0.3143882 , 0.20902523,

In [ ]:
from keras.models import load_model
model.save('kerassave')
newmodel = load_model('kerassave')